<center>

**Vježba 5 - Izlaganje modela kroz flask mikroservis + sigurnost (keycloak)**

</center>

U ovoj vježbi izlaženo model kroz osnovni omotač kojim se realizira mikroservis. U tu se svrhu koristi python *flask* modul.  
Dodatno, u vježbi će se dodati osnovna autentifikacija i autorizacija nad mikroservisom korištenjem OIDC-a. OIDC server koji će se koristiti je *keycloak*.  

Prvo instaliramo nužne python module:
- *flask* - modul za servisne omotače
- *flask_restx* - REST nadogradnja za *flask* modul
- *pyopenssl* - opcionalni modul koji omogućava SSL nad mikroservisom (https)

Potreban je i dodatni modul koji omogućava OIDC podršku za flask. To je modul *flask-oidc* koji ćemo preuzeti direktno sa github-a.

In [ ]:
%%bash
pip install flask flask_restx pyopensll
pip install git+https://github.com/puiterwijk/flask-oidc.git@refs/pull/144/head

S obzirom da ćemo koristiti kontejneriziranu verziju *keycloak*-a, moramo kreirati *persistent store* za taj OIDC serveru u koji će se spremati sve definicije.  
Sjetimo se da su *docker* kontejneri *stateless* pa bi nam svako gašenje ili restartanje *keycloak* serverskog kontejnera rezultiralo brisanjem svi definicija koje smo prethodno upisali.  

S obzirom da na Ubuntu operacijskom sustavu imamo PostgreSQL bazu podataka (koju smo instalirali na početku) - koristimo tu vanjsku bazu podataka kao persistent store.  
Prvo kreiramo bazu za *keycloak*.

In [ ]:
%%bash
export PGPASSWORD=pgadmin
createdb --username=postgres keycloak_db

U ```./keycloak``` folderu upisujemo *Dockerfile* u kojem imamo definiciju za generiranje custom *keycloak* docker image-a.  
U docker definiciji definiramo vanjsku bazu podataka ```jdbc:postgresql://localhost:5432/keycloak_db```, te definiramo *username* i *password* za tu bazu podataka.

In [ ]:
%%bash
mkdir keycloak

In [ ]:
%%writefile ./keycloak/Dockerfile
FROM quay.io/keycloak/keycloak:latest as builder

# Enable health and metrics support
ENV KC_HEALTH_ENABLED=true
ENV KC_METRICS_ENABLED=true

# Configure a database vendor
ENV KC_DB=postgres

WORKDIR /opt/keycloak
# for demonstration purposes only, please make sure to use proper certificates in production instead
# RUN keytool -genkeypair -storepass password -storetype PKCS12 -keyalg RSA -keysize 2048 -dname "CN=server" -alias server -ext "SAN:c=DNS:localhost,IP:127.0.0.1" -keystore conf/server.keystore
RUN /opt/keycloak/bin/kc.sh build

FROM quay.io/keycloak/keycloak:latest
COPY --from=builder /opt/keycloak/ /opt/keycloak/

# change these values to point to a running postgres instance
ENV KC_DB=postgres
ENV KC_DB_URL=jdbc:postgresql://localhost:5432/keycloak_db
ENV KC_DB_USERNAME=postgres
ENV KC_DB_PASSWORD=pgadmin
ENV KC_HOSTNAME=localhost
ENTRYPOINT ["/opt/keycloak/bin/kc.sh"]

Nakon toga generiramo svoj custom *keycloak* image kao *keycloak:latest*.  
Nakon generiranja ovaj image možemo pogledati u docker pluginu vscode-a.

In [ ]:
%%bash
cd keycloak
docker build . -t keycloak:latest

Startamo *keycloak:latest* image, te mu dajemo inicijalnog administracijskog korisnika.  
Ovako startan *keycloak* je u development modu, čime su neke provjere isključene (recimo forsiranje SSL tunela).  

In [ ]:
%%bash
docker run --net=host -e KEYCLOAK_ADMIN=admin -e KEYCLOAK_ADMIN_PASSWORD=admin keycloak:latest start-dev

Otvorimo administracijsko sučelje *keycloak* servera. Sjetimo se kako smo podesiti inicijalnog administracijskog korisnika kod pokretanja docker image-a.  

Nakon prijave, korištenjem *keycloak* administracijskog sučelja definiramo:
- Dodamo sigurnosnu domenu (*realm*) naziva ```aris```
- U sigurnosnu domenu ```aris``` dodamo klijenta ```flask``` sa sljedećim postavkama:
    - ```name = 'Flask Client'```
    - ```root URL = 'http://localhost:6789'```
    - ```valid redirection URLs = 'http://localhost:6789/*'```
    - ```client authentication = On```
    - ```Standard flow = True```
    - ```Direct access grant = True```
    - U Roles tabu dodamo ulogu ```flask_role```
    - Iz Credentials taba pokupimo ```Client secret``` ključ
- U sigurnosnu domenu ```aris``` dodamo grupu ```flask_users``` i mapiramo ju na ulogu ```flask_role```
- U sigurnosnu domenu ```aris``` dodamo nekog korisnika i mapiramo ga na grupu ```flask_users```

In [ ]:
import webbrowser
webbrowser.open("http://localhost:8080/admin")

Sljedeći kod pokazuje osnovni primjer u *flask*-u.  
Koristimo GET metodu bez parametara kako bismo vratili standardni ```plain/text``` s 'Hello, World!' tekstom.  
Nakon pokretanja sljedećeg odsječka, potrebno je ostaviti odsječak u ***run*** modu, a u browseru ručno otvoriti url _http://localhost:6789/test_  

Nakon što smo provjerili da je mikroservis aktivan, odsječak možemo stopirati.


In [ ]:
from flask import Flask
from flask_restx import Resource, Api

app = Flask("testapp")
api = Api(app)

@api.route('/test')
class TestApp(Resource):
    def get(self):
        return 'Hello, World!'
    
if __name__ == "__main__":
    app.run(host="localhost", debug=False, port=6789)

Zatim se prebacujemo na stvaranje omotača za postojeći model u *MLflow* serveru.  
Prvo definiramo REST sučelje.  

U samoj klasi REST omotača imamo ```_load``` metodu koja učitava IRIS model iz *MLflow* servera. Uz model učitava se i *scaler* objekt za transformaciju.  
Koristimo POST metodu za REST mikroservis. S obzirom da će sljedeća ćelija blokirati jupyter notebook, testiranje možemo izvesti s alatom poput postman-a.

In [ ]:
from flask import Flask, request
from flask_restx import Resource, Api, fields
import pandas as pd

import mlflow
import mlflow.sklearn
from mlflow.client import MlflowClient
import pickle

app = Flask("IrisModel")
api = Api(app)

iris_data = api.model(
  'Iris Data', {
    "sepal_length": fields.Float(description = "Sepal Length", required = True),
    "sepal_width": fields.Float(description = "Sepal Width", required = True),
    "petal_length": fields.Float(description = "Petal Length", required = True),
    "petal_width": fields.Float(description = "Petal Width", required = True)
  }
)

@api.route("/iris")
@api.expect(iris_data)
class IrisMSHandler(Resource):
    def _load_model(self):
        app.logger.info('Loading scikit-learn iris model from mlflow')
        mlflow.set_tracking_uri("http://localhost:5000")
        exps = mlflow.search_experiments(filter_string="name LIKE '%Iris%'")
        exp_id = exps[0].experiment_id

        client = MlflowClient()
        models = client.get_latest_versions("Iris MLP model", stages=["None"])
        self.mlp = mlflow.sklearn.load_model(models[0].source)
        run_id = models[0].run_id
        mlflow.artifacts.download_artifacts(artifact_uri="runs:/"+run_id+"/sklearn-model/scaler.pkl",dst_path="./")
        with open('scaler.pkl','rb') as f:
          self.scaler = pickle.load(f)

    def post(self):
        if not hasattr(self, "mlp"): self._load_model()
        idata = request.get_json()
        pdidata = pd.DataFrame(idata, index=[0])
        pdidata_s = self.scaler.transform(pdidata)
        pdidata_s = pd.DataFrame(pdidata_s)
        predict = self.mlp.predict(pdidata_s)
        return int(predict[0])

if __name__ == "__main__":
    app.run(host="localhost", debug=False, port=6789)

Želimo zaštititi mikroservis autentifikacijom i autorizacijom korištenjem prethodno instaliranog *keycloak* servera.  
Prvo definiramo sigurnosni opisnik. Bitno je upisati parametre koje smo prethodno definirali kroz administracijsko sučelje *keycloak* servera.  
Posebno se to odnosu na id klijenta, kao i *Client secret* koji je potreban za autentifikaciju klijenta - sjetimo se da klijent nije isto što i korisnik.  

In [ ]:
%%writefile client_secrets.json
{
    "web": {
        "issuer": "http://localhost:8080/realms/aris",
        "auth_uri": "http://localhost:8080/realms/aris/protocol/openid-connect/auth",
        "client_id": "flask",
        "client_secret": "baZvG6euJGvrkA1pODCVxPck82MO0T38",
        "redirect_uris": [
            "http://localhost:6789/*"
        ],
        "userinfo_uri": "http://localhost:8080/realms/aris/protocol/openid-connect/userinfo", 
        "token_uri": "http://localhost:8080/realms/aris/protocol/openid-connect/token",
        "token_introspection_uri": "http://localhost:8080/realms/aris/protocol/openid-connect/token/introspect"
    }
}

Stvorimo mali mikroservis koji osiguramo korištenjem *flask_oidc* python modula.  
Pokazujemo na prethodno stvoreni sigurnosni opisnik  ```client_secrets.json```.
Metodu GET anotiramo tako da je za nju potreban prijavljen korisnik.  
S obzirom da se radi o osnovnom OIDC toku, očekujemo redirekciju na OIDC server.  

Kao rezultat vraćamo ime i prezime korisnika.

In [ ]:
from flask import Flask
from flask_restx import Resource, Api
from flask_oidc import OpenIDConnect

app = Flask("securetest")
app.config.update({
    'SECRET_KEY': 'test',
    'OIDC_CLIENT_SECRETS': 'client_secrets.json',
    'OIDC_ID_TOKEN_COOKIE_SECURE': False,
    'OIDC_REQUIRE_VERIFIED_EMAIL': False,
    'OIDC_USER_INFO_ENABLED': True,
    'OIDC_OPENID_REALM': 'aris',
    'OIDC_SCOPES': ['openid', 'profile'],
    'OIDC_INTROSPECTION_AUTH_METHOD': 'client_secret_post'
})
oidc = OpenIDConnect(app)
api = Api(app)


@api.route('/sectest')
class SecureTestApp(Resource):
    @oidc.require_login
    def get(self):
        return 'Hello, '+oidc.user_getfield('name')
    
if __name__ == "__main__":
    app.run(host="localhost", debug=False, port=6789)